# Download responses and analyze them with DuckDB

In [1]:
%pip install --upgrade pip duckdb-engine faker sqlalchemy jupysql --quiet
%pip install ../.. --quiet  # or pip install citric

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path

from faker import Faker

import citric

## Create citric client

In [3]:
# Use your own server's parameters here
client = citric.Client(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
)

## Load survey file

In [4]:
with Path("../../examples/survey.lss").open("rb") as f:
    survey_id = client.import_survey(f)

## Generate fake response data

In [5]:
fake = Faker()

fake_responses = []

for _ in range(100):
    start_date = fake.date_time_this_month()
    submit_date = fake.date_time_between_dates(start_date)

    response = {
        "G01Q01": fake.text(max_nb_chars=100),
        "G01Q02": fake.random_int(1, 5),
        "token": fake.sha1()[:5],
        "ipaddr": fake.ipv4(),
        "startdate": start_date.isoformat(),
        "submitdate": submit_date.isoformat(),
    }

    fake_responses.append(response)

len(fake_responses)

100

## Activate survey and load fake responses

In [6]:
client.activate_survey(survey_id)
client.activate_tokens(survey_id)

result = client.add_responses(survey_id, fake_responses)
len(result)

100

## Export responses to a CSV file

In [7]:
with Path("responses.csv").open("wb") as file:
    file.write(client.export_responses(survey_id, file_format="csv"))

## Import the CSV into DuckDB

In [8]:
%load_ext sql

In [9]:
%sql duckdb://

In [10]:
%sql CREATE TABLE responses AS SELECT * FROM 'responses.csv'

*  duckdb://
Done.


Count
100


## Query the responses with DuckDB

In [11]:
%sql SELECT * FROM responses LIMIT 5

*  duckdb://
Done.


id,submitdate,lastpage,startlanguage,seed,token,startdate,datestamp,ipaddr,G01Q02,G01Q01,G02Q03,G02Q03[filecount]
1,2023-03-07 18:23:10,,en,,1eef3,2023-03-04 06:33:57,2023-03-10 03:29:09,20.134.168.36,2,Process necessary leader deep. Race size out production pass.,,
2,2023-03-06 04:08:01,,en,,b2d77,2023-03-03 18:12:55,2023-03-10 03:29:09,38.118.119.251,2,Entire nearly prove see else include service.,,
3,2023-03-06 05:32:40,,en,,94eab,2023-03-02 14:16:59,2023-03-10 03:29:09,65.248.2.179,3,Prove information finish floor party. City second one. Deal never box second top building.,,
4,2023-03-09 12:35:57,,en,,c03f3,2023-03-07 03:23:15,2023-03-10 03:29:09,159.145.242.156,3,Least class side local sure room would. Class view share soon. Strong have impact myself.,,
5,2023-03-09 04:04:11,,en,,19614,2023-03-09 02:20:27,2023-03-10 03:29:10,210.80.74.72,5,Popular something task discover. Six exactly source spend last.,,


### Show tally for multiple choice question

In [12]:
%sql SELECT G01Q02, count(1) as TOTAL FROM responses GROUP BY G01Q02

*  duckdb://
Done.


G01Q02,TOTAL
1,15
2,22
3,24
4,16
5,23


### Show which respondents took the longest to submit their responses

In [13]:
%sql SELECT token, submitdate - startdate AS duration FROM responses ORDER BY 2 DESC LIMIT 10

*  duckdb://
Done.


token,duration
73f22,"8 days, 11:36:09"
88776,"7 days, 18:01:13"
8702e,"7 days, 7:57:42"
cc1e5,"6 days, 21:10:36"
784f3,"5 days, 20:46:05"
9bf3a,"5 days, 11:32:52"
13d11,"5 days, 4:06:41"
2a1b3,"4 days, 22:08:44"
18b1c,"4 days, 16:13:30"
518f7,"4 days, 13:50:48"
